# Quantum Teleportation

In [1]:
from pyquil.quil import Program
from pyquil import api
from pyquil.gates import X, Z, H, CNOT

In [2]:
def make_bell_pair(q1, q2):
    """Makes a bell pair between qubits q1 and q2
    """
    return Program(H(q1), CNOT(q1, q2))


def teleport(start_index, end_index, ancilla_index):
    """Teleport a qubit from start to end using an ancilla qubit
    """
    program = make_bell_pair(end_index, ancilla_index)

    ro = program.declare("ro", memory_size=3)

    # do the teleportation
    program.inst(CNOT(start_index, ancilla_index))
    program.inst(H(start_index))

    # measure the results and store them in classical registers [0] and [1]
    program.measure(start_index, ro[0])
    program.measure(ancilla_index, ro[1])

    program.if_then(ro[1], X(2))
    program.if_then(ro[0], Z(2))

    program.measure(end_index, ro[2])

    print(program)
    return program

In [3]:
qvm = api.QVMConnection()

# initialize qubit 0 in |1>
teleport_demo = Program(X(0))
teleport_demo += teleport(0, 2, 1)
print("Teleporting |1> state: {}".format(qvm.run(teleport_demo, [2])))

# initialize qubit 0 in |0>
teleport_demo = Program()
teleport_demo += teleport(0, 2, 1)
print("Teleporting |0> state: {}".format(qvm.run(teleport_demo, [2])))

# initialize qubit 0 in |+>
teleport_demo = Program(H(0))
teleport_demo += teleport(0, 2, 1)
print("Teleporting |+> state: {}".format(qvm.run(teleport_demo, [2], 10)))

H 2
CNOT 2 1
DECLARE ro BIT[3]
CNOT 0 1
H 0
MEASURE 0 ro[0]
MEASURE 1 ro[1]
JUMP-WHEN @THEN1 ro[1]
JUMP @END2
LABEL @THEN1
X 2
LABEL @END2
JUMP-WHEN @THEN3 ro[0]
JUMP @END4
LABEL @THEN3
Z 2
LABEL @END4
MEASURE 2 ro[2]

Teleporting |1> state: [[1]]
H 2
CNOT 2 1
DECLARE ro BIT[3]
CNOT 0 1
H 0
MEASURE 0 ro[0]
MEASURE 1 ro[1]
JUMP-WHEN @THEN1 ro[1]
JUMP @END2
LABEL @THEN1
X 2
LABEL @END2
JUMP-WHEN @THEN3 ro[0]
JUMP @END4
LABEL @THEN3
Z 2
LABEL @END4
MEASURE 2 ro[2]

Teleporting |0> state: [[0]]
H 2
CNOT 2 1
DECLARE ro BIT[3]
CNOT 0 1
H 0
MEASURE 0 ro[0]
MEASURE 1 ro[1]
JUMP-WHEN @THEN1 ro[1]
JUMP @END2
LABEL @THEN1
X 2
LABEL @END2
JUMP-WHEN @THEN3 ro[0]
JUMP @END4
LABEL @THEN3
Z 2
LABEL @END4
MEASURE 2 ro[2]

Teleporting |+> state: [[0], [1], [1], [1], [0], [0], [1], [0], [0], [0]]
